# TensorFlow 2.0 alpha - Saving & Restoring Models
## MNIST Dataset

#### Model progress can be saved during Training, as well as after - the model can pause and resume to avoid long training times - This also allows for the model to be shared with others
#### Different TF model APIs require different saving methods - This exercise uses tf.keras API

In [1]:
!pip install -q h5py pyyaml

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install msgpack

    100% |████████████████████████████████| 122kB 1.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for msgpack ... done
  Stored in directory: /Users/MPHA/Library/Caches/pip/wheels/e0/eb/73/79c4057260fcb51c5f12cee027dda5cf79b92b618a82529c74
Successfully built msgpack
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from __future__ import absolute_import, division, print_function

import os
import tensorflow as tf
from tensorflow import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Retrieve Dataset - Demonstrate Saving Weights
#### Use only the first 1000 examples from the MNIST dataset

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data() 

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28)/255.0
test_images = test_images[:1000].reshape(-1, 28 * 28)/255.0

11493376/11490434 [==============================] - 11s 1us/step


### Define a Simple Model

In [5]:
# Return a short Sequential model

def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    return model

# Create a basic model Instance

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save Checkpoints during Training
#### Typical usage involves automatically saving checkpoints During training, and at the End - the model can be used at a later time without retraining it, and pick up where it left off if training was interrupted 

In [7]:
# Train model and pass the ModelCheckpoint Callback

checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                               save_weights_only=True,
                                               verbose=1)
model = create_model()

model.fit(train_images, train_labels, epochs=10,
         validation_data = (test_images, test_labels),
         callbacks = [cp_callback])         # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.2674 - accuracy: 0.6274
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 694us/sample - loss: 1.1700 - accuracy: 0.6540 - val_loss: 0.7452 - val_accuracy: 0.7750
Epoch 2/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.4607 - accuracy: 0.8646
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 462us/sample - loss: 0.4423 - accuracy: 0.8690 - val_loss: 0.5537 - val_accuracy: 0.8370
Epoch 3/10
 768/1000 [======================>.......] - ETA: 0s - loss: 0.3035 - accuracy: 0.9219
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 443us/sample - loss: 0.2952 - accuracy: 0.9230 - val_loss: 0.4688 - val_accuracy: 0.8510
Epoch 4/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.2202 - accuracy: 0.9469
Epoch

### This creates a single collection of TF Checkpoint files - updated after each epoch

In [8]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


### Create an untrained model - Restore from the Weights
#### If restoring a model from Weights alone, must use a model of the same architecture

In [9]:
# Create Untrained model

model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print('Untrained model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 141us/sample - loss: 2.3714 - accuracy: 0.1120
Untrained model, accuracy: 11.20%


In [10]:
# Load the Weights

model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 109us/sample - loss: 0.4204 - accuracy: 0.8680
Restored model, accuracy: 86.80%


#### Once again - 86.80 % accuracy achieved
## Checkpoint Callback Options - Names & Frequency
#### Train a New Model using these Options

In [12]:
# Train a new model - saving uniquely Named Checkpoints, every 5 epochs

checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'    # include epoch in file name
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                verbose=1,
                                                save_weights_only=True,   # save weights
                                                period=5)                 # every 5 epochs
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
         epochs = 50, callbacks = [cp_callback],
         validation_data = (test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


#### Of the Resulting Checkpoints above - Choose the Latest one

In [13]:
!ls {checkpoint_dir}

checkpoint                       cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001 cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index               cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001 cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index               cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001 cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index               cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001 cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index               cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001 cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index               cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

### Test - reset the Model, Loading the Latest Checkpoint

In [16]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)

print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 156us/sample - loss: 0.4834 - accuracy: 0.8770
Restored model, accuracy: 87.70%


## Manually Save Weights

In [17]:
# save the weights

model.save_weights('./checkpoints/my_checkpoint')

# restore the weights

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 159us/sample - loss: 0.4834 - accuracy: 0.8770
Restored model, accuracy: 87.70%


## Save the Entire Model
#### The model configuration, weights, and variables can be saved, allowing access without having the original python code - Models can be loaded in Tensorflow.js (HDF5, saved model), then train them to be run in Web Browsers or convert them to run on Mobile devices using Tensorflow LIte (HDF5, saved model)

### Keras provides basic save format using the HDF5 standard

In [18]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# save entire model to a HDF5 file

model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 363us/sample - loss: 1.1440 - accuracy: 0.6700
Epoch 2/5
1000/1000 [==============================] - 0s 278us/sample - loss: 0.4149 - accuracy: 0.8860
Epoch 3/5
1000/1000 [==============================] - 0s 258us/sample - loss: 0.2846 - accuracy: 0.9200
Epoch 4/5
1000/1000 [==============================] - 0s 238us/sample - loss: 0.2042 - accuracy: 0.9520
Epoch 5/5
1000/1000 [==============================] - 0s 234us/sample - loss: 0.1525 - accuracy: 0.9700


### Recreate the Model from that File - including weights & optimizer

In [19]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Check the Accuracy

loss, acc = new_model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 172us/sample - loss: 0.4438 - accuracy: 0.8500
Restored model, accuracy: 85.00%


### Keras saves the Weight values, Model configuration, and Optimizer configuration by inspecting model architecture - it is NOT currently able to save TF optimizers (from tf.train)

## save_model - an additional (experimental) method

In [21]:
# Build a fresh model

model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 432us/sample - loss: 1.1478 - accuracy: 0.6810
Epoch 2/5
1000/1000 [==============================] - 0s 277us/sample - loss: 0.4260 - accuracy: 0.8840
Epoch 3/5
1000/1000 [==============================] - 0s 305us/sample - loss: 0.2807 - accuracy: 0.9250
Epoch 4/5
1000/1000 [==============================] - 0s 297us/sample - loss: 0.1986 - accuracy: 0.9560
Epoch 5/5
1000/1000 [==============================] - 0s 224us/sample - loss: 0.1475 - accuracy: 0.9710


#### Create a saved_model - place it in time-stamped directory

In [22]:
import time
saved_model_path = './saved_models/{}'.format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0403 22:57:23.624114 140736985473984 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0403 22:57:23.625437 140736985473984 tf_logging.py:161] Export includes no default signature!
W0403 22:57:24.460882 140736985473984 tf_logging.py:161] Export includes no default signature!


'./saved_models/1554357442'

In [23]:
# List saved models

!ls saved_models/

1554357442


#### Reload a new Keras model from the Saved Model

In [24]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


#### Run the Restored Model

In [25]:
model.predict(test_images).shape

(1000, 10)

In [29]:
# Model needs to be Compiled before evaluation - not necessary with deployment alone

new_model.compile(optimizer=model.optimizer,
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

# Evaluate restored model

loss, acc = new_model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 180us/sample - loss: 0.4351 - accuracy: 0.8590
Restored model, accuracy: 85.90%


# Additional Notes
#### Entire models can be saved as HDF5 files, or by using saved_model method
#### Weights and Checkpoints can also be saved

In [30]:
# support for this exercise can be found at tensorflow.org - Permissions granted
#
#@title MIT License
#
# Copyright (c) 2017 François Chollet